## Training models with BRAILS

In this notebook, we include the code for training pretrained models for BRAILS. Users can also provide their own datasets to train models. 

In the first step, we need to install pytorch-image-models which includes several pretrained models on ImageNet that we will use:

In [ ]:
pip install git+https://github.com/rwightman/pytorch-image-models

  Cloning https://github.com/rwightman/pytorch-image-models to /tmp/pip-req-build-_bvqtxxa
  Running command git clone -q https://github.com/rwightman/pytorch-image-models /tmp/pip-req-build-_bvqtxxa
  Created wheel for timm: filename=timm-0.5.5-py3-none-any.whl size=432818 sha256=58390a6dde078f6085878eac0810215284def7a855446dc343acda9d5dd2d1ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-jllrx1af/wheels/69/3d/b0/be55cbadabd87a0e1875d63c7492d199097a39cc2433637650
Successfully built timm


Next the dataset needs to be uploaded via Google drive. Please refer to https://neptune.ai/blog/google-colab-dealing-with-files for instructions.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import necessary packages for training models,

In [ ]:
import argparse
import datetime
import os
import matplotlib as mpl
import numpy as np

mpl.use('Agg')
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import confusion_matrix

############################
# Import vision transformer that we will fine-tune

from timm.models import vision_transformer, resnet
############################

Set the random seeds such that the experiments are reproducible,

In [ ]:
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)
np.random.seed(1337)


Define the transformations that we will use,

In [ ]:
def get_transform():

    # normalize the datasets
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    # transformatins used during training to improve the model perfomrance
    train_transforms = [
        transforms.RandomResizedCrop(224, scale=(0.7, 1.)),
        transforms.RandomGrayscale(p=0.5),
        transforms.ColorJitter(0.5, 0.5, 0.5, 0.5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        normalize]


    train_transforms = transforms.Compose(train_transforms)

    val_transforms = [transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize]

    val_transforms = transforms.Compose(val_transforms)

    return train_transforms, val_transforms


 The code below is used for training and testing the model,

In [ ]:
def parse( model, data_loader, criterion, optimizer, parse_type, epoch, epochs, device):
   
    train = False
    if parse_type == 'train':
        
        train = True
        model.train()
    else:

        model.eval()

    predictions = []
    ground_truths = []
    correct = 0

    avg_loss = 0

    for i, (images, labels ) in enumerate(data_loader):

        images = images.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        output = model(images)

        loss = criterion(output, labels)

        if train:
            loss.backward()
            optimizer.step()

        avg_loss += loss.detach().cpu().numpy()

        if (i + 1) % 5 == 0:
            print("Epoch [%d/%d], Iter [%d/%d] Loss: %.4f" % (
                epoch + 1, epochs, i + 1, len(data_loader), loss.data.item()))

 
        _, pred = torch.max(output, 1)

        predictions.extend(pred.detach().cpu().tolist())  # Should do the detach internally
        ground_truths.extend(labels.detach().cpu().tolist())
    
        correct += pred.eq(labels.view_as(pred)).sum().item()


    cf = confusion_matrix(ground_truths, predictions)

    print ("Confusion matrix:")
    print (cf)

    cls_cnt = cf.sum(axis=1)
    print (cls_cnt)
    cls_hit = np.diag(cf)
    cls_acc = cls_hit / cls_cnt

    out_cls_acc = '%s Class Accuracy: %s'%(parse_type, (np.array2string(cls_acc, max_line_width=np.inf, separator=',', formatter={'float_kind':lambda x: "%.3f" % x})))
    print(out_cls_acc)

    print ('%s Accuracy: %s'% (parse_type, 100.0 * correct / len(data_loader.dataset)))

    print ('%s Loss: %s'% (parse_type, avg_loss / len(data_loader)))

    ##############################

    return avg_loss / len(data_loader)

    ##############################


In [ ]:
def main_work(model='resnet18', train_dataset, test_dataset, num_classes, save_dir, lr, batch_size, epochs, test_only):

    #########################################################################
    # Load vision transformer

    if model == 'resnet18'
      model_prefix = "resnet18"

      #model = vision_transformer.vit_base_patch16_224(pretrained=True)

      model = resnet.resnet18(pretrained=True)
      # change the number of classes 
    
    else:

      print ("In this notebook, we use resnet18 as an example.")
      
      return
    
    model.reset_classifier(num_classes)

    #########################################################################

    pytorch_total_params = sum(p.numel() for p in model.parameters())

    trianable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print ("Total parameters: ", pytorch_total_params)
    print ("Trianable parameters: ", trianable_params)

    
    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9)

    #########################################################################

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model = model.to(device)

    #########################################################################

    criterion = nn.CrossEntropyLoss()

    best_checkpoint =  save_dir + "best_" + model_prefix + "_rooftype.pkl" 

    # class to index
    print (train_dataset.class_to_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle=True, num_workers=4)

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= batch_size, shuffle=False, num_workers=4)
    
    #########################################################################

    if test_only:

        best_checkpoint =  save_dir + "best_" + model_prefix + "_rooftype.pkl" 
            
        ######################################

        state_dict = torch.load(best_checkpoint)

        model.load_state_dict(state_dict)

        _ = parse(model, test_loader, criterion, optimizer, 'test', 0, 0, device=device)

    else:

        #######################################################################
        # Learning rate decay
        scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

        best_perf = -1e6
        lowest_loss = 1e6

        for epoch in range(epochs):
            
            model.train()
            #############################################################
            
            print('Training epoch: {}'.format(epoch))
            
            avg_train_loss = parse(model, train_loader, criterion, optimizer, 'train', epoch, epochs, device=device)
            
            scheduler.step()

            print ("save model")
            print (best_checkpoint)
            torch.save(model.state_dict(), best_checkpoint)



In [ ]:
def Get_RoofTypeImages(transforms, root_dir="/content/gdrive/MyDrive/model_training/roofType"):


    data = torchvision.datasets.ImageFolder(root=root_dir, transform=transforms)

    return data


Define the hyperparameters for training，

In [ ]:
# Number of classes of the training dataset
num_classes = 3

# The learning rate used for optimization
lr = 0.01 

batch_size = 256

epochs = 100

test_only = False
#########################################################
# Prepare the dataset

# Data directory. Changed this as needed. 
root_dir = "/content/gdrive/MyDrive/model_training/roofType"

# Directory for saving the trained model
save_dir = "/content/gdrive/MyDrive/model_training/"

train_transforms, val_transforms = get_transform()

train_dataset =  Get_RoofTypeImages(train_transforms, root_dir)

# The same dataset but with val_transforms for testing
test_dataset =  Get_RoofTypeImages(val_transforms, root_dir)
#########################################################

In [ ]:
main_work(model="resnet18", train_dataset, test_dataset, num_classes, save_dir, lr, batch_size, epochs, test_only)

Total parameters:  11178051
Trianable parameters:  11178051
{'flat': 0, 'gabled': 1, 'hipped': 2}
Training epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [1/100], Iter [5/24] Loss: 0.8114
Epoch [1/100], Iter [10/24] Loss: 0.5298
Epoch [1/100], Iter [15/24] Loss: 0.5097
Epoch [1/100], Iter [20/24] Loss: 0.5378
Confusion matrix:
[[1140  611  249]
 [ 494 1318  188]
 [ 164  168 1668]]
[2000 2000 2000]
train Class Accuracy: [0.570,0.659,0.834]
train Accuracy: 68.76666666666667
train Loss: 0.6548929003377756
save model
/content/gdrive/MyDrive/model_training/best_transformer_rooftype.pkl
Training epoch: 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [2/100], Iter [5/24] Loss: 0.4567
Epoch [2/100], Iter [10/24] Loss: 0.4712
Epoch [2/100], Iter [15/24] Loss: 0.3518
Epoch [2/100], Iter [20/24] Loss: 0.3678
Confusion matrix:
[[1454  478   68]
 [ 395 1539   66]
 [  81   74 1845]]
[2000 2000 2000]
train Class Accuracy: [0.727,0.769,0.922]
train Accuracy: 80.63333333333334
train Loss: 0.44074773664275807
save model
/content/gdrive/MyDrive/model_training/best_transformer_rooftype.pkl
Training epoch: 2


...

Epoch [98/100], Iter [5/24] Loss: 0.0047
Epoch [98/100], Iter [10/24] Loss: 0.0147
Epoch [98/100], Iter [15/24] Loss: 0.0348
Epoch [98/100], Iter [20/24] Loss: 0.0069
Confusion matrix:
[[1985   15    0]
 [  15 1983    2]
 [   2    1 1997]]
[2000 2000 2000]
train Class Accuracy: [0.993,0.992,0.999]
train Accuracy: 99.41666666666667
train Loss: 0.014435439137741923
save model
/content/gdrive/MyDrive/model_training/best_transformer_rooftype.pkl
Training epoch: 98


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [99/100], Iter [5/24] Loss: 0.0220
Epoch [99/100], Iter [10/24] Loss: 0.0030
Epoch [99/100], Iter [15/24] Loss: 0.0086
Epoch [99/100], Iter [20/24] Loss: 0.0046
Confusion matrix:
[[1988   10    2]
 [  13 1987    0]
 [   1    1 1998]]
[2000 2000 2000]
train Class Accuracy: [0.994,0.994,0.999]
train Accuracy: 99.55
train Loss: 0.01451891363831237
save model
/content/gdrive/MyDrive/model_training/best_transformer_rooftype.pkl
Training epoch: 99


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [100/100], Iter [5/24] Loss: 0.0052
Epoch [100/100], Iter [10/24] Loss: 0.0092
Epoch [100/100], Iter [15/24] Loss: 0.0231
Epoch [100/100], Iter [20/24] Loss: 0.0107
Confusion matrix:
[[1990    8    2]
 [  11 1988    1]
 [   5    0 1995]]
[2000 2000 2000]
train Class Accuracy: [0.995,0.994,0.998]
train Accuracy: 99.55
train Loss: 0.014062009130915007
save model
/content/gdrive/MyDrive/model_training/best_transformer_rooftype.pkl
